In [1]:
from pyspark.sql import SparkSession  # Import the SparkSession class

spark = SparkSession.builder \
    .appName("MyApp") \
    .getOrCreate()

print("Spark Version : " + spark.version)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/18 22:31:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/18 22:31:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Spark Version : 3.5.1
